In [1]:
# most of the samples that fail QC were sequenced at sanger, and all the high SNP samples are sanger. 
# The samples at sanger were sequenced in batched - do we have a dodgy batch?

import hail as hl
import pyspark
from hail.plot import show
hl.plot.output_notebook()

from bokeh.plotting import output_file, save

sc = pyspark.SparkContext()
tmp_dir = "hdfs://spark-master:9820/"
lustre_dir = "file:///lustre/scratch123/qc/"
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference="GRCh38")

Loading BokehJS ...

2022-03-17 10:52:07 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/backend/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/backend/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.1.2
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.88-7d39379870da
LOGGING: writing to /home/ubuntu/jupyter/hail-20220317-1052-0.2.88-7d39379870da.log


In [4]:
mtdir = lustre_dir + "matrixtables/"

htfile = mtdir + "mt_pops_QC_filters.ht"
ht = hl.read_table(htfile)

ht = ht.annotate(filter_fail_count=(hl.len(ht.qc_metrics_filters) - 2))

fail_count_expr = (hl.case()
                   .when(ht.filter_fail_count == 0, '0')
                   .when(ht.filter_fail_count == 1, '1')
                   .when(ht.filter_fail_count == 2, '2')
                   .when(ht.filter_fail_count == 3, '3')
                   .when(ht.filter_fail_count == 4, '4')
                   .default("")
                  )
ht = ht.annotate(filter_fail_count_str = fail_count_expr).key_by('s')

seq_expr = (hl.case()
                   .when(ht.s.startswith('EGAN'), 'Sanger')
                   .when(ht.s.startswith('Z'), 'Bristol')
                   .default("")
                  )
ht = ht.annotate(sequencing_location = seq_expr).key_by('s')
ht.show()


+-------------------+----------+----------+-----------+--------------+
| s                 | callrate |   f_stat | is_female | assigned_pop |
+-------------------+----------+----------+-----------+--------------+
| str               |  float64 |  float64 |      bool | str          |
+-------------------+----------+----------+-----------+--------------+
| "EGAN00003143328" | 1.00e+00 | 8.30e-01 |     False | "EUR"        |
| "EGAN00003143329" | 1.00e+00 | 8.26e-01 |     False | "EUR"        |
| "EGAN00003143330" | 1.00e+00 | 8.75e-01 |     False | "AFR"        |
| "EGAN00003143331" | 1.00e+00 | 8.39e-01 |     False | "EUR"        |
| "EGAN00003143334" | 1.00e+00 | 8.51e-01 |     False | "EUR"        |
| "EGAN00003143336" | 1.00e+00 | 8.81e-01 |     False | "EUR"        |
| "EGAN00003143337" | 1.00e+00 | 8.57e-01 |     False | "EUR"        |
| "EGAN00003143338" | 1.00e+00 | 8.20e-01 |     False | "EUR"        |
| "EGAN00003143339" | 1.00e+00 | 8.71e-01 |     False | "EUR"        |
| "EGAN00003143340" | 1.00e+00 | 8.57e-01 |     False | "EUR"        |
+-------------------+----------+----------+-----------+--------------+

+---------------------+--------------------+------------------------+
| sample_qc.call_rate | sample_qc.n_called | sample_qc.n_not_called |
+---------------------+--------------------+------------------------+
|             float64 |              int64 |                  int64 |
+---------------------+--------------------+------------------------+
|            1.00e+00 |             552164 |                      8 |
|            1.00e+00 |             552167 |                      5 |
|            1.00e+00 |             552169 |                      3 |
|            1.00e+00 |             552170 |                      2 |
|            1.00e+00 |             552168 |                      4 |
|            1.00e+00 |             552166 |                      6 |
|            1.00e+00 |             552172 |                      0 |
|            1.00e+00 |             552167 |                      5 |
|            1.00e+00 |             552169 |                      3 |
|            1.00e+00 |             552171 |                      1 |
+---------------------+--------------------+------------------------+

+----------------------+---------------------+-----------------+
| sample_qc.n_filtered | sample_qc.n_hom_ref | sample_qc.n_het |
+----------------------+---------------------+-----------------+
|                int64 |               int64 |           int64 |
+----------------------+---------------------+-----------------+
|                    0 |              535825 |           11111 |
|                    0 |              535303 |           11701 |
|                    0 |              534327 |           12536 |
|                    0 |              535491 |           11499 |
|                    0 |              535424 |           11588 |
|                    0 |              535584 |           11499 |
|                    0 |              536251 |           10806 |
|                    0 |              535534 |           11525 |
|                    0 |              535983 |           11019 |
|                    0 |              535620 |           11390 |
+----------------------+---------------------+-----------------+

+---------------------+---------------------+-----------------------+
| sample_qc.n_hom_var | sample_qc.n_non_ref | sample_qc.n_singleton |
+---------------------+---------------------+-----------------------+
|               int64 |               int64 |                 int64 |
+---------------------+---------------------+-----------------------+
|                5228 |               16339 |                     0 |
|                5163 |               16864 |                     0 |
|                5306 |               17842 |                     0 |
|                5180 |               16679 |                     0 |
|                5156 |               16744 |                     0 |
|

In [5]:
#metadata_file = lustre_dir + '/resources/all_samples_with_proceed_and_seq_info_and_warehouse_info.txt'
metadata_file = lustre_dir + '/resources/all_samples_with_proceed_and_seq_info_and_warehouse_info_egas_from_mlwh.txt'
#full metadata file doesn't parse properly
#metadata_file = lustre_dir + '/resources/batches.txt'
metadata_ht = hl.import_table(metadata_file, delimiter="\t").key_by('ega')
metadata_ht.show()


2022-03-17 10:52:50 Hail: INFO: Loading 30 fields. Counts by type:
  str: 30


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
sangersampleid,suppliersamplename,sample_sent,cohort,qlet,filenum,trio,dup_sample,cidB3565,id.check,DNA.Source,cid.qlet,cellline.sample.with.blood.dup,blood.sample.with.cellline.dup,Proceed.x,batch.x,Proceed.y,batch.y,name,id_run,manual_qc,mean_bait_coverage,verify_bam_id_score,sequencing_location,haplotype_calling,ega,sex,sequencescape_supplier_id,cram,missing_from_sequencescape
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Z20057""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""Bristol""","""Yes""","""""","""""","""""","""/lustre/scratch119/humgen/projects/birth_cohort_wes/alspac/external_cram/remapped/Z20057.sorted.dups.coord.cram""",""""""
"""Z02306""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""Bristol""","""Yes""","""""","""""","""""","""/lustre/scratch119/humgen/projects/birth_cohort_wes/alspac/external_cram/remapped/Z02306.sorted.dups.coord.cram""",""""""
"""Z18774""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""Bristol""","""Yes""","""""","""""","""""","""/lustre/scratch119/humgen/projects/birth_cohort_wes/alspac/external_cram/remapped/Z18774.sorted.dups.coord.cram""",""""""
"""Z02380""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""Bristol""","""Yes""","""""","""""","""""","""/lustre/scratch119/humgen/projects/birth_cohort_wes/alspac/external_cram/remapped/Z02380.sorted.dups.coord.cram""",""""""
"""Z21288""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""Bristol""","""Yes""","""""","""""","""""","""/lustre/scratch119/humgen/projects/birth_cohort_wes/alspac/external_cram/remapped/Z21288.sorted.dups.coord.cram""",""""""
"""Z07632""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""Bristol""","""Yes""","""""","""""","""""","""/lustre/scratch119/humgen/projects/birth_cohort_wes/alspac/external_cram/remapped/Z07632.sorted.dups.coord.cram""",""""""
"""Z15043""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""Bristol""","""Yes""","""""","""""","""""","""/lustre/scratch119/humgen/projects/birth_cohort_wes/alspac/external_cram/remapped/Z15043.sorted.dups.coord.cram""",""""""
"""Z04742""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""Bristol""","""Yes""","""""","""""","""""","""/lustre/scratch119/humgen/projects/birth_cohort_wes/alspac/external_cram/remapped/Z04742.sorted.dups.coord.cram""",""""""


In [6]:
metadata_ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'sangersampleid': str 
    'suppliersamplename': str 
    'sample_sent': str 
    'cohort': str 
    'qlet': str 
    'filenum': str 
    'trio': str 
    'dup_sample': str 
    'cidB3565': str 
    'id.check': str 
    'DNA.Source': str 
    'cid.qlet': str 
    'cellline.sample.with.blood.dup': str 
    'blood.sample.with.cellline.dup': str 
    'Proceed.x': str 
    'batch.x': str 
    'Proceed.y': str 
    'batch.y': str 
    'name': str 
    'id_run': str 
    'manual_qc': str 
    'mean_bait_coverage': str 
    'verify_bam_id_score': str 
    'sequencing_location': str 
    'haplotype_calling': str 
    'ega': str 
    'sex': str 
    'sequencescape_supplier_id': str 
    'cram': str 
    'missing_from_sequencescape': str 
----------------------------------------
Key: ['ega']
----------------------------------------


In [7]:
ht = ht.annotate(batch = metadata_ht[ht.s]['batch.y'])
ht.show()

2022-03-17 10:54:15 Hail: INFO: Ordering unsorted dataset with network shuffle  


+-------------------+----------+----------+-----------+--------------+
| s                 | callrate |   f_stat | is_female | assigned_pop |
+-------------------+----------+----------+-----------+--------------+
| str               |  float64 |  float64 |      bool | str          |
+-------------------+----------+----------+-----------+--------------+
| "EGAN00003143328" | 1.00e+00 | 8.30e-01 |     False | "EUR"        |
| "EGAN00003143329" | 1.00e+00 | 8.26e-01 |     False | "EUR"        |
| "EGAN00003143330" | 1.00e+00 | 8.75e-01 |     False | "AFR"        |
| "EGAN00003143331" | 1.00e+00 | 8.39e-01 |     False | "EUR"        |
| "EGAN00003143334" | 1.00e+00 | 8.51e-01 |     False | "EUR"        |
| "EGAN00003143336" | 1.00e+00 | 8.81e-01 |     False | "EUR"        |
| "EGAN00003143337" | 1.00e+00 | 8.57e-01 |     False | "EUR"        |
| "EGAN00003143338" | 1.00e+00 | 8.20e-01 |     False | "EUR"        |
| "EGAN00003143339" | 1.00e+00 | 8.71e-01 |     False | "EUR"        |
| "EGAN00003143340" | 1.00e+00 | 8.57e-01 |     False | "EUR"        |
+-------------------+----------+----------+-----------+--------------+

+---------------------+--------------------+------------------------+
| sample_qc.call_rate | sample_qc.n_called | sample_qc.n_not_called |
+---------------------+--------------------+------------------------+
|             float64 |              int64 |                  int64 |
+---------------------+--------------------+------------------------+
|            1.00e+00 |             552164 |                      8 |
|            1.00e+00 |             552167 |                      5 |
|            1.00e+00 |             552169 |                      3 |
|            1.00e+00 |             552170 |                      2 |
|            1.00e+00 |             552168 |                      4 |
|            1.00e+00 |             552166 |                      6 |
|            1.00e+00 |             552172 |                      0 |
|            1.00e+00 |             552167 |                      5 |
|            1.00e+00 |             552169 |                      3 |
|            1.00e+00 |             552171 |                      1 |
+---------------------+--------------------+------------------------+

+----------------------+---------------------+-----------------+
| sample_qc.n_filtered | sample_qc.n_hom_ref | sample_qc.n_het |
+----------------------+---------------------+-----------------+
|                int64 |               int64 |           int64 |
+----------------------+---------------------+-----------------+
|                    0 |              535825 |           11111 |
|                    0 |              535303 |           11701 |
|                    0 |              534327 |           12536 |
|                    0 |              535491 |           11499 |
|                    0 |              535424 |           11588 |
|                    0 |              535584 |           11499 |
|                    0 |              536251 |           10806 |
|                    0 |              535534 |           11525 |
|                    0 |              535983 |           11019 |
|                    0 |              535620 |           11390 |
+----------------------+---------------------+-----------------+

+---------------------+---------------------+-----------------------+
| sample_qc.n_hom_var | sample_qc.n_non_ref | sample_qc.n_singleton |
+---------------------+---------------------+-----------------------+
|               int64 |               int64 |                 int64 |
+---------------------+---------------------+-----------------------+
|                5228 |               16339 |                     0 |
|                5163 |               16864 |                     0 |
|                5306 |               17842 |                     0 |
|                5180 |               16679 |                     0 |
|                5156 |               16744 |                     0 |
|

In [8]:
#subset to sanger only
ht = ht.filter(ht.sequencing_location == "Sanger")
ht.count()

12004

In [9]:
#scatter plot number SNPs vs heterozygosity rate coloured by batch
p = hl.plot.scatter(ht.sample_qc.n_snp, ht.sample_qc.heterozygosity_rate, xlabel='Number of SNPs', ylabel='Heterozygosity rate', title='Number of SNPs vs heterozygosity rate Sanger only, coloured by batch number', label=ht.batch, collect_all = True)
show(p)

plot_file = "/lustre/scratch123/qc//plots/sample_qc/hetrate_snps_batches.html"
output_file(plot_file)
save(p)

2022-03-17 10:54:52 Hail: INFO: Ordering unsorted dataset with network shuffle  


'/lustre/scratch123/qc/plots/sample_qc/hetrate_snps_batches.html'

In [10]:
#count numbers per batch for all sanger samples
ht.aggregate(hl.agg.counter(ht.batch))

2022-03-17 10:56:09 Hail: INFO: Ordering unsorted dataset with network shuffle  


frozendict({'batch1': 1099, 'batch10': 688, 'batch11': 1663, 'batch2': 1295, 'batch3': 985, 'batch4': 1129, 'batch5': 929, 'batch6': 1349, 'batch7': 746, 'batch8': 930, 'batch9': 1181, None: 10})

In [11]:
#numbers which fail QC per batch
ht.aggregate(hl.agg.filter(ht.filter_fail_count_str != '0', hl.agg.counter(ht.batch)))

2022-03-17 10:56:30 Hail: INFO: Ordering unsorted dataset with network shuffle1]


frozendict({'batch1': 253, 'batch10': 118, 'batch11': 441, 'batch2': 412, 'batch3': 185, 'batch4': 157, 'batch6': 189, 'batch7': 1, 'batch9': 111})

In [12]:
#numbers which pass QC per batch
ht.aggregate(hl.agg.filter(ht.filter_fail_count_str == '0', hl.agg.counter(ht.batch)))

2022-03-17 10:56:39 Hail: INFO: Ordering unsorted dataset with network shuffle


frozendict({'batch1': 846, 'batch10': 570, 'batch11': 1222, 'batch2': 883, 'batch3': 800, 'batch4': 972, 'batch5': 929, 'batch6': 1160, 'batch7': 745, 'batch8': 930, 'batch9': 1070, None: 10})

In [13]:
#samples with no batch annotation - these are the control samples
ht_nobatch = ht.filter(hl.is_defined(ht.batch), keep = False)
ht_nobatch.show()

2022-03-17 10:57:03 Hail: INFO: Ordering unsorted dataset with network shuffle  


+----------------------------+----------+-----------+-----------+--------------+
| s                          | callrate |    f_stat | is_female | assigned_pop |
+----------------------------+----------+-----------+-----------+--------------+
| str                        |  float64 |   float64 |      bool | str          |
+----------------------------+----------+-----------+-----------+--------------+
| "EGAN00003332049"          | 1.00e+00 | -4.97e-02 |      True | "EUR"        |
| "EGAN00003332049_remapped" | 1.00e+00 | -5.17e-02 |      True | "EUR"        |
| "EGAN00003332050"          | 1.00e+00 |  1.29e-02 |      True | "EUR"        |
| "EGAN00003332050_remapped" | 1.00e+00 |  1.29e-02 |      True | "EUR"        |
| "EGAN00003332051"          | 1.00e+00 |  8.65e-01 |     False | "EUR"        |
| "EGAN00003332051_remapped" | 1.00e+00 |  8.65e-01 |     False | "EUR"        |
| "EGAN00003332052"          | 1.00e+00 |  8.61e-01 |     False | "EAS"        |
| "EGAN00003332052_remapped" | 1.00e+00 |  8.61e-01 |     False | "EAS"        |
| "EGAN00003332053"          | 1.00e+00 |  8.39e-01 |     False | "EUR"        |
| "EGAN00003332053_remapped" | 1.00e+00 |  8.39e-01 |     False | "EUR"        |
+----------------------------+----------+-----------+-----------+--------------+

+---------------------+--------------------+------------------------+
| sample_qc.call_rate | sample_qc.n_called | sample_qc.n_not_called |
+---------------------+--------------------+------------------------+
|             float64 |              int64 |                  int64 |
+---------------------+--------------------+------------------------+
|            1.00e+00 |             552169 |                      3 |
|            1.00e+00 |             552169 |                      3 |
|            1.00e+00 |             552166 |                      6 |
|            1.00e+00 |             552166 |                      6 |
|            1.00e+00 |             552166 |                      6 |
|            1.00e+00 |             552166 |                      6 |
|            1.00e+00 |             552169 |                      3 |
|            1.00e+00 |             552169 |                      3 |
|            1.00e+00 |             552172 |                      0 |
|            1.00e+00 |             552172 |                      0 |
+---------------------+--------------------+------------------------+

+----------------------+---------------------+-----------------+
| sample_qc.n_filtered | sample_qc.n_hom_ref | sample_qc.n_het |
+----------------------+---------------------+-----------------+
|                int64 |               int64 |           int64 |
+----------------------+---------------------+-----------------+
|                    0 |              538242 |            8766 |
|                    0 |              538239 |            8767 |
|                    0 |              538078 |            8982 |
|                    0 |              538075 |            8984 |
|                    0 |              538196 |            8862 |
|                    0 |              538196 |            8863 |
|                    0 |              538680 |            7867 |
|                    0 |              538673 |            7875 |
|                    0 |              538110 |            8925 |
|                    0 |              538107 |            8928 |
+----------------------+---------------------+-----------------+

+---------------------+---------------------+-----------------------+
| sample_qc.n_hom_var | sample_qc.n_non_ref | sample_qc.n_singleton |
+---------------------+---------------------+-----------------------+
|               int64 |               int64 |                 int64 |
+---------------------+---------------------+-----------------------+
|                5161 |               13927 |                     0 |
|                5163 |               13930 |                     0 |
|                5106 |               14088 |      